In [7]:
import numpy as np
import sys
import os
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
import project
import importlib


In [34]:
p = Project.load("../slidearound.proj")

In [35]:
p

In [43]:
from session_utils import SESSION_OBJ_2D, SESSION_EVENTS
print(p.sessions[0][SESSION_EVENTS])

[{'label': ['Stella Artois', 'Shell'], 'end': 33, 'start': 0}, {'label': ['Shell', 'Stella Artois'], 'end': 67, 'start': 44}, {'label': ['Stella Artois', 'Shell'], 'end': 119, 'start': 82}, {'label': ['Shell', 'Stella Artois'], 'end': 156, 'start': 136}, {'label': ['Stella Artois', 'Shell'], 'end': 189, 'start': 167}, {'label': ['Shell', 'Stella Artois'], 'end': 214, 'start': 197}, {'label': ['Stella Artois', 'Shell'], 'end': 243, 'start': 224}, {'label': ['Shell', 'Stella Artois'], 'end': 278, 'start': 254}, {'label': ['Stella Artois', 'Shell'], 'end': 308, 'start': 288}, {'label': ['Shell', 'Stella Artois'], 'end': 340, 'start': 319}, {'label': ['Shell', 'Stella Artois'], 'end': 367, 'start': 352}, {'label': ['Stella Artois', 'Shell'], 'end': 396, 'start': 381}, {'label': ['Stella Artois', 'Shell'], 'end': 418, 'start': 404}, {'label': ['Shell', 'Stella Artois'], 'end': 445, 'start': 427}, {'label': ['Stella Artois', 'Shell'], 'end': 468, 'start': 450}, {'label': ['Stella Artois', 'S

In [57]:
importlib.reload(project)

<module 'project' from 'D:\\git\\learn-to-perform\\project.py'>

In [58]:
p = Project.load("../slidearound.proj")

In [59]:
p.generate_data()

NameError: name 'project' is not defined

In [60]:
p.sessions[0][]

In [66]:
random_indices = np.arange(5)
np.random.shuffle(random_indices)

In [67]:
random_indices

array([4, 1, 0, 2, 3])